In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import os
import cv2
import keras
import keras.utils as utils
from keras.models import Sequential
from keras import layers
from keras import models
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras import regularizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
import tensorflow as tf
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import seaborn as sns

In [ ]:
IMG_HEIGHT = 256
IMG_WIDTH = 256
batch_size = 32
epochs = 15
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
train_dir = r"C:\Users\Marcus\Documents\NYP\NYPY3\Y3S2\Hackathon\HacX\train"
test_dir = r"C:\Users\Marcus\Documents\NYP\NYPY3\Y3S2\Hackathon\HacX\test"

In [ ]:
train_image_generator = ImageDataGenerator(
                    rescale=1./255, rotation_range=25,
                    width_shift_range=.25, height_shift_range=.25,
                    horizontal_flip=True,  zoom_range=0.2)

validation_image_generator = ImageDataGenerator(rescale=1./255)

train_data_gen = train_image_generator.flow_from_directory(batch_size=batch_size,
                                                           directory=train_dir,
                                                           shuffle=True,
                                                           target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                           class_mode='categorical')

test_data_gen = validation_image_generator.flow_from_directory(batch_size=batch_size,
                                                              directory=test_dir,
                                                              shuffle = False,
                                                              target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                              class_mode='categorical')

In [ ]:
from keras.applications.mobilenet_v2 import MobileNetV2
from keras.layers import GlobalAveragePooling2D

#modify input_shape and classifier layers
base_model=MobileNetV2(weights='imagenet',include_top=False, input_shape=(IMG_HEIGHT, IMG_WIDTH, 3))

x = base_model.output
# x = Flatten()(x)
x=GlobalAveragePooling2D()(x)
x=Dense(512,activation='relu')(x)
x = Dropout(0.2)(x)  
preds=Dense(3, activation='softmax')(x)

Trans_Model=Model(inputs=base_model.input, outputs=preds)

In [ ]:
base_model.summary()

In [ ]:
#freeze base model layers
base_model.trainable = False

# Let's confirm all the layers of convolutional base are frozen.
for layer in base_model.layers:
    print(layer.name, layer.trainable)

In [ ]:
path_checkpoint= r"C:\Users\Marcus\Documents\NYP\NYPY3\Y3S2\Hackathon\Project Data\model_checkpoint.weights.h5"
es_callback = keras.callbacks.EarlyStopping(monitor="accuracy",patience=4)
modelckpt_callback=keras.callbacks.ModelCheckpoint(
    monitor="accuracy",
    filepath=path_checkpoint,
    verbose=1,
    save_weights_only=True,
    save_best_only=True)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
Trans_Model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
history = Trans_Model.fit(train_data_gen,epochs=epochs,validation_data=test_data_gen, callbacks=[es_callback, modelckpt_callback])

In [ ]:
loss = history.history['loss']
val_loss = history.history['val_loss']
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

plt.figure(figsize=(20, 8))
plt.subplot(1, 2, 1)
plt.plot(loss,label='loss')
plt.plot(val_loss,label='val_loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(acc,label='acc')
plt.plot(val_acc,label='val_acc')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.show()

In [ ]:
# Predict on the test dataset
y_pred_prob = Trans_Model.predict(test_data_gen)  # Predictions as probabilities
y_pred = np.argmax(y_pred_prob, axis=1)         # Convert probabilities to class indices
y_true = test_data_gen.classes                  # True labels from the test generator

# Get class labels
class_indices = test_data_gen.class_indices
class_labels = list(class_indices.keys())

# Confusion Matrix
cm = confusion_matrix(y_true, y_pred)

# Classification Report
report = classification_report(y_true, y_pred, target_names=class_labels)
accuracy = accuracy_score(y_true, y_pred)

# Print Classification Report
print("Classification Report:")
print(report)
print(f"Overall Accuracy: {accuracy * 100:.2f}%")

# Plot Confusion Matrix
plt.figure(figsize=(8, 6))
cm_df = pd.DataFrame(cm, index=class_labels, columns=class_labels)  # Create a DataFrame for better readability
sns.heatmap(cm_df, annot=True, fmt='d', cmap='Blues', cbar=False)   # Plot the heatmap
plt.title('Confusion Matrix')
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.show()

FINETUNING

In [ ]:
#Define freeze and unfrozen layers
for layer in Trans_Model.layers[:20]:
  layer.trainable=False

for layer in Trans_Model.layers[20:]:
  layer.trainable=True

In [ ]:
path_checkpoint= r"C:\Users\Marcus\Documents\NYP\NYPY2\NYP Y2S2\EGT214 Deep Learning\Project Data\model_checkpoint.weights.h5"
es_callback = keras.callbacks.EarlyStopping(monitor="accuracy",patience=4)
modelckpt_callback=keras.callbacks.ModelCheckpoint(
    monitor="accuracy",
    filepath=path_checkpoint,
    verbose=1,
    save_weights_only=True,
    save_best_only=True)
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)
Trans_Model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
history = Trans_Model.fit(train_data_gen,epochs=epochs,validation_data=test_data_gen, callbacks=[es_callback, modelckpt_callback])

In [ ]:
loss = history.history['loss']
val_loss = history.history['val_loss']
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

plt.figure(figsize=(20, 8))
plt.subplot(1, 2, 1)
plt.plot(loss,label='loss')
plt.plot(val_loss,label='val_loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(acc,label='acc')
plt.plot(val_acc,label='val_acc')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.show()

In [ ]:
# Predict on the test dataset
y_pred_prob = Trans_Model.predict(test_data_gen)  # Predictions as probabilities
y_pred = np.argmax(y_pred_prob, axis=1)         # Convert probabilities to class indices
y_true = test_data_gen.classes                  # True labels from the test generator

# Get class labels
class_indices = test_data_gen.class_indices
class_labels = list(class_indices.keys())

# Confusion Matrix
cm = confusion_matrix(y_true, y_pred)

# Classification Report
report = classification_report(y_true, y_pred, target_names=class_labels)
accuracy = accuracy_score(y_true, y_pred)

# Print Classification Report
print("Classification Report:")
print(report)
print(f"Overall Accuracy: {accuracy * 100:.2f}%")

# Plot Confusion Matrix
plt.figure(figsize=(8, 6))
cm_df = pd.DataFrame(cm, index=class_labels, columns=class_labels)  # Create a DataFrame for better readability
sns.heatmap(cm_df, annot=True, fmt='d', cmap='Blues', cbar=False)   # Plot the heatmap
plt.title('Confusion Matrix')
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.show()